In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler as ss
from sklearn.utils import shuffle

In [2]:
data = pd.read_csv("./water_potability.csv").dropna()[:2000]
print(len(data))
features = shuffle(data)
features.head()

2000


,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
207,7.947486,269.809407,34440.773434,6.958283,390.104223,425.627814,13.749261,56.635811,3.700854,0
292,8.291299,211.217034,20008.134957,8.767875,275.767321,475.909130,12.614669,66.612984,3.424702,1
1845,8.176080,201.553701,21856.988628,5.241047,320.260120,464.803330,15.627641,61.654744,4.112322,0
3200,8.581043,153.709092,19970.434294,7.661687,312.885127,520.808359,14.781584,74.127793,2.883145,1
12,7.119824,156.704993,18730.813653,3.606036,282.344050,347.715027,15.929536,79.500778,3.445756,0


In [3]:
labels = features.pop("Potability")
features = np.array(features)
features.shape
labels = np.array(labels).reshape(-1,1)

In [4]:
features = ss().fit_transform(features)


In [5]:
features = torch.tensor(features, dtype=torch.float32)
labels = torch.tensor(labels, dtype=torch.float32)

In [6]:
from torch import sigmoid
model = torch.nn.Sequential(
    torch.nn.Linear(9,6),
    torch.nn.ReLU(),
    torch.nn.Linear(6,3),
    torch.nn.ReLU(),
    torch.nn.Linear(3,1),
    torch.nn.Sigmoid()
)

In [13]:
loss_fn = torch.nn.BCELoss()
opt = torch.optim.SGD(model.parameters(), lr=0.01)

In [14]:
epochs = 1000000

for epoch in range(epochs):
    preds = model(features)
    loss = loss_fn(preds, labels)

    loss.backward()

    opt.step()
    opt.zero_grad()

    with torch.no_grad():
        if (epoch+1)%(epochs//10) == 0:
            right = 0
            for i in range(len(preds)):
                if preds[i][0] >= 0.5:
                    preds[i][0] = 1
                else:
                    preds[i][0] = 0

                if preds[i][0] == labels[i][0]:
                    right += 1


            print(f"Loss : {loss}")
            print(f"Accuracy : {round(right * 100/ len(preds), 2)}%")


Loss : 0.5528362989425659
Accuracy : 71.25%
Loss : 0.5528362989425659
Accuracy : 71.25%
Loss : 0.5528362393379211
Accuracy : 71.25%
Loss : 0.5528362393379211
Accuracy : 71.25%
Loss : 0.5528362989425659
Accuracy : 71.25%
Loss : 0.5528362393379211
Accuracy : 71.25%
Loss : 0.5528362989425659
Accuracy : 71.25%


KeyboardInterrupt: 